In [372]:
import pandas as pd
import numpy as np
from scipy import stats

In [373]:
df = pd.read_csv("data.csv")
df["Species"] = np.where(df["Species"] == "Dog", 1, 0)

In [374]:
X = df.drop("Species", axis = 1)

for col in X.columns:
    X[col] = (X[col] - X[col].mean()) / X[col].std()
    #X[col] = stats.zscore(X[col])
    
y = df["Species"]
X.insert(0, "Intercept", 1)

In [375]:
X

,Intercept,Height,Weight
0,1,1.489381,1.153415
1,1,1.811009,0.944967
2,1,-0.044533,0.342782
3,1,-0.366160,1.130254
4,1,0.153391,-0.792104
5,1,-1.603188,-1.834347
6,1,-0.143495,-0.676300
7,1,-0.440382,-0.792104
8,1,-0.737269,-0.027793
9,1,-0.118755,0.551231


## <font color = "brown"> Calculate First Derivative </font>

In [376]:
def ProbHat(x, b):
    
    # Calculate predicted probability given X and beta
    Xtb = np.dot(x, b)
    eXtb = np.exp(-Xtb)
    return 1 / (1 + eXtb)

In [377]:
def PartialIndRow(yi, xi, beta, xij):
    
    # Calculate the partial derivative value for each row
    return xij * (yi - ProbHat(xi, beta))

In [378]:
def PartialIndBeta(y, X, beta, j):
    
    # Calculating derivatives Log-Likelihood with respect to beta-j
    xj = X.iloc[:,j]
    PartialList = [PartialIndRow(y, x, beta, j) for y, x, j in zip(y.to_numpy(), X.to_numpy(), xj)]
    
    return sum(PartialList)

In [379]:
def PartialLL(y, X, beta):
    
    partials = []
    # Loop through each parameter
    for j in range(len(beta)):
        
        # Append the partial derivative of the Log-Likelihood
        # w.r.t to each parameter
        partials.append(PartialIndBeta(y, X, beta, j))
        
    return np.array(partials, dtype = np.float64)
    

In [380]:
PartialLL(y, X, [1,1,1])

array([-1.61343515,  0.98858098,  0.40125501])

## <font color = "brown"> Calculate Second Derivative (Jacobian) </font>

In [381]:
def PartialPartialIndRow(y, xi, beta, xij, xik):
    
    # Calculate the double partial derivative value for each row
    return (-xij * xik) * ProbHat(xi, beta) * (1 - ProbHat(xi, beta))

In [382]:
def PartialPartialTwoBeta(y, X, beta, j, k):
    
    # Calculating double derivatives of the Log-Likelihood 
    # with respect to beta-j and beta-k
    xj = X.iloc[:,j]
    xk = X.iloc[:,k]
    PartialList = [PartialPartialIndRow(y, x, beta, j, k) for y, x, j, k in zip(y.to_numpy(), X.to_numpy(), xj, xk)]
    
    return sum(PartialList)

In [383]:
def PartialPartialLL(y, X, beta):
    
    Jacobi = np.zeros([3,3])
    
    # Loop through each parameter as row
    for j in range(len(beta)):
        
        # Loop through each parameter as column
        for k in range(len(beta)):
            
            # The Jacobian is symmetric,
            # We only need to calculate upper triangle
            if j <= k:
                
                # Calculate double derivatives
                PartialValues = PartialPartialTwoBeta(y, X, beta, j, k)
                Jacobi[j][k] = PartialValues
                
                # Insert for lower triangle
                if j != k:
                    Jacobi[k][j] = PartialValues
                
    return Jacobi      

## <font color = "brown"> Estimating Parameters using Newton's Method </font>

In [389]:
def NewtonEstimator(y, X, beta):
    i = 0
    while sum(abs(PartialLL(y, X, beta))) > 1e-6:
        
        # First derivative
        DLL = PartialLL(y, X, beta)
        
        # Second derivative
        DDLL = PartialPartialLL(y, X, beta)
        print(beta)
        DDLLInv = np.linalg.inv(DDLL)
        AdditionFactor = np.matmul(DDLLInv, DLL)
        beta = beta + AdditionFactor
        #print(sum(abs(PartialLL(y, X, beta))))
        
        i+=1
        
    return beta

In [1]:
#NewtonEstimator(y, X, [0.019, 0.88, 0.65])